In [1]:
import objaverse
import random
uids = objaverse.load_uids()
# print(len(uids)) == 798759
annotations = objaverse.load_annotations(uids[:508759])

'''
Annotation in form :
{uid: {
    'uri' : '',
    'tags' : [{'name' : '',}, {'name' : '',}, {'name' : '',}]
}}
'''
# get me count of all categories (eg. names) in the annotations
from collections import defaultdict
categories = defaultdict(int)

for uid, annotation in annotations.items():
    for tag in annotation['tags']:
        categories[tag['name']] += 1

# sort the categories by count and print if category has more than 100 annotations
sorted_categories = sorted(categories.items(), key=lambda x: x[1], reverse=True)
# limit to categories with more than 100 annotations
sorted_categories = [cat for cat in sorted_categories if cat[1] > 100]
for cat in sorted_categories:
    print(cat)

    

In [4]:
import objaverse
import random

def get_categories_given_uid(uid):
    annotations = objaverse.load_annotations([uid])
    return [item['name'] for item in annotations[uid]['tags']]

In [ ]:
import time


def download_3d_objects_from_objectverse(nr_objects, categories = []):
    '''
    saves to "~/.objaverse/hf-objaverse-v1/glbs/" path by default
    '''
    assert type(categories) == list, "categories should be a list of strings"
    uids = objaverse.load_uids()
    uids_to_download = []
    start_time = time.time()
    TIMEOUT = 5 * 60
    while len(uids_to_download) < nr_objects or time.time() - start_time > TIMEOUT:
        # get 10 uids and filter them out
        uids = random.sample(uids, 10)
        uids = [uid for uid in uids if set(categories).issubset(get_categories_given_uid(uid))]
        # filter uids that are already in uids_to_download
        uids = [uid for uid in uids if uid not in uids_to_download]


    objects = objaverse.load_objects(
        uids=uids_to_download,
        download_processes=4
    )
    return objects

In [18]:
import random
import objaverse
import os
import multiprocessing

PROCESSES = multiprocessing.cpu_count() - 2 # leave some cpus for other stuff to not kill the PC
OBJAVERSE_PATH = "/Users/szczekulskij/.objaverse/hf-objaverse-v1/glbs/" # default path for where the objaverse objects are stored
IMAGE_PATH = '''/Users/szczekulskij/masters-work/spring '24/CSE-252D/objaverse-rendering/objaverse_data''' # default path for where the objaverse images are stored

# if IMAGE_PATH or OBJAVERSE_PATH is not found, it will be created
if not os.path.exists(IMAGE_PATH): os.makedirs(IMAGE_PATH)
if not os.path.exists(OBJAVERSE_PATH): os.makedirs(OBJAVERSE_PATH)

def get_ids_of_already_downloaded_objects(objaverse_path = OBJAVERSE_PATH, image_path = IMAGE_PATH):
    ''' 
    Objects from objaverse are stored in form PATH/{gibberish}/{uid}.glb
    Rendering (2d images) are stored in form PATH/{uid}/{light_params_camera_params}.png
    '''
    downloaded_objects = set()
    for root, dirs, files in os.walk(OBJAVERSE_PATH):
        for file in files:
            if file.endswith(".glb"):
                uid = file.split(".")[0]
                downloaded_objects.add(uid)

    for root, dirs, files in os.walk(IMAGE_PATH):
        for file in files:
            if file.endswith(".png"):
                uid = file.split("/")[0]
                downloaded_objects.add(uid)

    return downloaded_objects
    


def download_3d_objects_from_objectverse(nr_objects, nr_processes = PROCESSES):
    '''
    saves to "~/.objaverse/hf-objaverse-v1/glbs/" path by default
    '''
    allUids = objaverse.load_uids()
    downloaded_uids = get_ids_of_already_downloaded_objects()
    uids = [uid for uid in allUids if uid not in downloaded_uids] # remove already downloaded objects (quick operation, dw)
    uids = random.sample(uids, nr_objects) # get random objects
    objects = objaverse.load_objects(
        uids=uids,
        download_processes=nr_processes
    )
    return objects

In [19]:
import argparse
import json
import math
import os
import random
import sys
import time
import urllib.request
import uuid
from typing import Tuple
from mathutils import Vector, Matrix
import numpy as np

import bpy
from mathutils import Vector
